# FAST-OAD - Multirotor Design

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging
from fastoad.utils.postprocessing import VariableViewer
from models.register import register_openmdao_systems
import shutil
register_openmdao_systems()

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, 'oad_process_drone_trajectory_2.toml')
SOURCE_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_trajectory.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In the configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the reference parameters from my_ref_values.xml as default values:

In [2]:
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)
#api.generate_inputs(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 34 )
INFO    : Installed bundle models (ID 33 )
INFO    : Problem inputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_inputs_trajectory.xml


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [3]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 34 )
INFO    : Installed bundle models (ID 33 )


INFO    : N2 diagram written in D:\THESE\Tools\FAST_OAD_Drone\workdir\n2.html


## 2. Multirotor MDA

Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [43]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_trajectory.xml')
api.variable_viewer(INPUT_FILE)

In [5]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 34 )
INFO    : Installed bundle models (ID 33 )
INFO    : Computation finished after 0.07 seconds
INFO    : Problem outputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_outputs.xml


In [6]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mda.xml')
shutil.copy(OUTPUT_FILE, MDA_OUTPUT_FILE)
api.variable_viewer(OUTPUT_FILE)

## 3. Multirotor MDO

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file.

In [7]:
api.optimization_viewer(CONFIGURATION_FILE)

In [63]:
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 34 )
INFO    : Installed bundle models (ID 33 )
INFO    : Computation finished after 0.83 seconds
INFO    : Problem outputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_outputs.xml


Optimization terminated successfully    (Exit mode 0)
            Current function value: [0.05341626]
            Iterations: 30
            Function evaluations: 31
            Gradient evaluations: 30
Optimization Complete
-----------------------------------


In [64]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')
shutil.copy(OUTPUT_FILE, MDO_OUTPUT_FILE)
api.optimization_viewer(CONFIGURATION_FILE)

In [65]:
api.variable_viewer(OUTPUT_FILE)

## 4. Analysis and plots

In [15]:
from utils.postprocessing.analysis_and_plots import *

#fig = drone_geometry_plot(MDA_OUTPUT_FILE, name='Drone MDA')
#fig = drone_geometry_plot(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)

fig = drone_geometry_plot(MDO_OUTPUT_FILE, name='Drone MDO')
fig.show()

In [12]:
fig = mass_breakdown_sun_plot_drone(OUTPUT_FILE)
fig.show()

In [13]:
fig = mass_breakdown_bar_plot_drone(MDA_OUTPUT_FILE, name='Drone MDA')
fig = mass_breakdown_bar_plot_drone(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)
fig.show()

In [14]:
fig = energy_breakdown_sun_plot_drone(OUTPUT_FILE)
fig.show()